In [2]:
import pulp

In [3]:
# 1. With exactly 2700 square inches of cardboard, 
# we wish to construct a box (width 2x, depth x, height 2x).
# We would like to maximize the volume V the box can hold.
#Which values of width, depth, and height fulfill our objective?

In [4]:
model = pulp.LpProblem("Volume maximising problem", pulp.LpMaximize)
width = pulp.LpVariable('width', lowBound=0, cat='Float')
depth = pulp.LpVariable('depth', lowBound=0, cat='Float')
height = pulp.LpVariable('height', lowBound=0, cat='Float')

In [5]:
# This is no longer LP :) Problem is convex though
#model += width * depth * height, "Volume"
"""
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-5-1cbe0e29754f> in <module>
      1 # This is no longer LP :) Problem is convex though
----> 2 model += width * depth * height, "Volume"

/usr/local/lib/python3.7/site-packages/pulp/pulp.py in __mul__(self, other)
    208 
    209     def __mul__(self, other):
--> 210         return LpAffineExpression(self) * other
    211 
    212     def __rmul__(self, other):

/usr/local/lib/python3.7/site-packages/pulp/pulp.py in __mul__(self, other)
    786                         e[v] = c * x
    787         elif isinstance(other,LpVariable):
--> 788             return self * LpAffineExpression(other)
    789         else:
    790             if other != 0:

/usr/local/lib/python3.7/site-packages/pulp/pulp.py in __mul__(self, other)
    774             if len(other):
    775                 if len(self):
--> 776                     raise TypeError("Non-constant expressions cannot be multiplied")
    777                 else:
    778                     c = self.constant

TypeError: Non-constant expressions cannot be multiplied
"""

TypeError: Non-constant expressions cannot be multiplied

In [6]:
import torch
import numpy as np
from torch.autograd import Variable
from torch import tensor
from itertools import combinations

In [7]:
device = torch.device('cpu')

In [76]:
# Note to self: I bumped into initialization, gradient explosion, lr schedule / skipping the minima 
# ...and this is not quite a neural network yet!
x_var = Variable(tensor(10.), requires_grad=True)

def loss_func(v):
    width = 2*v
    depth = v
    height = 2*v
    dims = [width, depth, height]
    
    volume = 1;
    for dim in dims:
        volume *= dim

    ## objective: max volume
    loss_v = -volume
    
    square = 0;
    for dim1, dim2 in combinations(dims, 2):
        square += dim1 * dim2;

    ## constraint: total square <= 2700
    loss_sq = min(0, 2700 - square)
    loss_sq = loss_sq**2

    ## constraint: dims are positive
    loss_dim_pos = min(0, -v)
        
    loss = loss_v + loss_sq + loss_dim_pos

    return loss, square, width, depth, height

square = 0;
vol = 0;
lr = 1e-4
for i in range(50):
    loss, square, width, depth, height = loss_func(x_var)
    loss.backward()
    with torch.no_grad():
        x_var -= lr * x_var.grad
        # otherwise these are collected
        x_var.grad.zero_()

        vol = (width * height * depth).item()
        sq = square.item()
    print(f"{i:03d}: w: {width:.3f}, h: {height:.3f}, d: {depth:.3f}, sq: {sq:.3f}, vol: {vol:.3f}")

000: w: 2.000, h: 2.000, d: 1.001, sq: 8.000, vol: 4.005
001: w: 2.003, h: 2.003, d: 1.003, sq: 8.021, vol: 4.021
002: w: 2.005, h: 2.005, d: 1.004, sq: 8.042, vol: 4.037
003: w: 2.008, h: 2.008, d: 1.005, sq: 8.063, vol: 4.052
004: w: 2.010, h: 2.010, d: 1.007, sq: 8.084, vol: 4.068
005: w: 2.013, h: 2.013, d: 1.008, sq: 8.105, vol: 4.084
006: w: 2.016, h: 2.016, d: 1.009, sq: 8.126, vol: 4.100
007: w: 2.018, h: 2.018, d: 1.010, sq: 8.147, vol: 4.116
008: w: 2.021, h: 2.021, d: 1.012, sq: 8.169, vol: 4.133
009: w: 2.024, h: 2.024, d: 1.013, sq: 8.190, vol: 4.149
010: w: 2.026, h: 2.026, d: 1.014, sq: 8.212, vol: 4.165
011: w: 2.029, h: 2.029, d: 1.016, sq: 8.233, vol: 4.182
012: w: 2.032, h: 2.032, d: 1.017, sq: 8.255, vol: 4.198
013: w: 2.034, h: 2.034, d: 1.018, sq: 8.277, vol: 4.215
014: w: 2.037, h: 2.037, d: 1.020, sq: 8.299, vol: 4.232
015: w: 2.040, h: 2.040, d: 1.021, sq: 8.320, vol: 4.248
016: w: 2.042, h: 2.042, d: 1.023, sq: 8.342, vol: 4.265
017: w: 2.045, h: 2.045, d: 1.0

In [180]:
# 3. A farmer wants to customize his fertilizer for his current crop.
# He can buy plant food mix A and plant food mix B. 
# Each cubic yard of food A contains 20 pounds of phosphoric acid, 30 pounds of nitrogen and 5 pounds of potash. 
# Each cubic yard of food B contains 10 pounds of phosphoric acid, 30 pounds of nitrogen and 10 pounds of potash.
# He requires a minimum of 460 pounds of phosphoric acid, 9060 pounds of nitrogen and 220 pounds of potash. 
# If food A costs $30 per cubic yard and food B costs $35 per cubic yard, 
# how many cubic yards of each food should the farmer blend to meet the minimum chemical requirements at a minimal cost?
# What is this cost?
model = pulp.LpProblem("Cost minimization problem", pulp.LpMinimize)
volumeA = pulp.LpVariable('volumeA', lowBound=0, cat='Float')
volumeB = pulp.LpVariable('volumeB', lowBound=0, cat='Float')
model += volumeA * 20 + volumeB * 30, "Cost"
model += 20*volumeA + 10*volumeB >= 460.0, "Phosphoric acid requirement"
model += 30*volumeA + 30*volumeB >= 9060.0, "Nitrogen requirement"
model += 5*volumeA + 10*volumeB >= 220.0, "Potash requirement"

In [181]:
model.solve()

1

In [182]:
pulp.LpStatus[model.status]

'Optimal'

In [183]:
pulp.value(model.objective)

6040.0

In [184]:
for v in model.variables():
    print(v.name, "=", v.varValue)

volumeA = 302.0
volumeB = 0.0
